In [1]:
### Version: 02.05.2023
#python -m pip install prophet

In [2]:
#;;*****************************************************************;;;
#;;*****************************************************************;;;
#;;;****************************************************************;;;
#;;;***  FIRMA          : PARADOX                                ***;;;
#;;;***  Autor          : ALEXANDer WAGNER                       ***;;;
#;;;***  STUDIEN-NAME   : ALMATY2023                             ***;;;
#;;;***  STUDIEN-NUMMER :                                        ***;;;
#;;;***  SPONSOR        :                                        ***;;;
#;;;***  ARBEITSBEGIN   : 01.04.2023                             ***;;;
#;;;****************************************************************;;;
#;;;*--------------------------------------------------------------*;;;
#;;;*---  PROGRAMM      : KZ2023A.IPYNB                         ---*;;;
#;;;*---  PROGRAMM0     : KSFE_2023Final2.IPYNB, 25.04.2025     ---*;;;
#;;;*---  Parent0       : +KSFE22_DASH2.IPYNB                   ---*;;;
#;;;*---  BESCHREIBUNG  : System                                ---*;;;
#;;;*---                :                                       ---*;;;
#;;;*---                :                                       ---*;;;
#;;;*---  VERSION   VOM : 02.05.2023                            ---*;;;
#;;;*--   KORREKTUR VOM :                                       ---*;;;
#;;;*--                 :                                       ---*;;;
#;;;*---  INPUT         :.INI                                   ---*;;;
#;;;*---  OUTPUT        :                                       ---*;;;
#;;;*--------------------------------------------------------------*;;;
#;;;************************ Änderung ******************************;;;
#;;;****************************************************************;;;
#;;;  Wann              :               Was                        *;;;
#;;;*--------------------------------------------------------------*;;;
#;;;* 02.05.2023        : Nachbesserung                            *;;;
#;;;*---                : Median + Propeth funktionieren!!!     ---*;;;
#;;;****************************************************************;;;

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import chart_studio.plotly as py 
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import flask
import json
from dash.dependencies import Input, Output
from datetime import datetime
import dash_table
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import pyperclip
from prophet import Prophet

C:\Users\satur\AppData\Local\Temp\ipykernel_16156\3729857354.py:33: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\satur\AppData\Local\Temp\ipykernel_16156\3729857354.py:34: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\satur\AppData\Local\Temp\ipykernel_16156\3729857354.py:48: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


In [3]:
cnx = sqlite3.connect('data/KZAPP.db')
df=pd.read_sql_query("Select ID1, ID, IDNAME, POP, MKN, FKN FROM KZ_ALL where AGEN=1", cnx) 
pop=pd.read_sql_query("Select ID, MKN, FKN FROM KZ_POP ORDER BY AGEN DESC", cnx) 

my_list=['< 5 лет','5 - 9 лет','10 - 14 лет','15 - 19 лет','20 - 24 лет','25 - 29 лет','30 - 34 лет','35 - 39 лет',
'40 - 44 лет','45 - 49 лет','50 - 54 лет','55 - 59 лет','60 - 64 лет','65 - 69 лет','70 - 74 лет','75 - 79 лет',
'80 - 84 лет','85+ лет']

map_obj = map(str.upper, my_list)
y = list(map_obj)

age_df = pd.read_csv('data/country_data_master.csv', 
                     usecols=lambda cols: 'perc' in cols or cols == 'country' or cols == 'median_age_total')
age_df = age_df.sort_values(['median_age_total'])
age_categories = ['0-14', '15-24', '25-54', '55-64', '65+']


df_stock= pd.read_sas(f'data/stock_price_FB.sas7bdat', encoding="latin-1")
df_pred = pd.read_sas(f'data/pred_prophet_FB.sas7bdat', encoding="latin-1")
FP =pd.read_sas('data/All_prophet_fb.sas7bdat', encoding="latin-1")
pyperclip.copy("KZ-11")

In [4]:
colors = ['red', '#9C0C38', 'orange']
tabs_styles = {"height": "30px"}

tab_style = {
    "borderBottom": "1px solid #d6d6d6",
    "padding": "2px",
    "fontWeight": "bold",
    "vertical-align": "middle",
    "backgroundColor": "#111111", 
}

tab_selected_style = {
    "borderTop": "1px solid #d6d6d6",
    "borderBottom": "1px solid #d6d6d6",
    "backgroundColor": "#111111",
    "color": "yellow",
    "padding": "5px",
    "font-size": 18,
}


In [5]:

app = JupyterDash(external_stylesheets=[dbc.themes.SLATE])
app.title = "Kazakhstan Dashboard"
app.layout = html.Div([
   html.Div(className="row", children=[ 
   html.Div(["", 
             dcc.Input(id='my-input', value='KZ-00', type='text', placeholder='', 
             style={'display':'inline-block', 'border': '1px solid black', "height": "60px", 
                    'background-color': 'black', 'border-color': 'black', 'color': 'black', 'width':'100px'})              
     ],style={"background": "black", 'width':'100px',"height": "60px", 'marginLeft':19,'marginRight':5}), 
      html.H1("Python Plotly: Dashboard II", style={"textAlign": "center", 'color': 'Yellow', 'fontSize': 40}),  
    ]
   ), 

 dcc.Tabs(id="tabs-with-classes", value='Table1', 
              parent_className='custom-tabs', 
              className='custom-tabs-container',
        children=[
            dcc.Tab(
                label="Демография",
                value="Table1",
                style=tab_style,
                selected_style=tab_selected_style,
                className='custom-tab',
                selected_className='custom-tab--selected'
            ),
            dcc.Tab(
                label="Популяция",
                value="Table2",
                style=tab_style,
                selected_style=tab_selected_style,
                className='custom-tab',
                selected_className='custom-tab--selected'
            ),
            dcc.Tab(
                label="Возрастное распределение по странам",
                value="Table3",
                style=tab_style,
                selected_style=tab_selected_style,
                className='custom-tab',
                selected_className='custom-tab--selected'
            ),
            dcc.Tab(
                label="Прогноз Заболеваемости",
                value="Table4",
                style=tab_style,
                selected_style=tab_selected_style,
                className='custom-tab',
                selected_className='custom-tab--selected'
            ),
         ],
         style=tabs_styles,
         colors={"border": "yellow", "primary": "red",
                 "background": "#111111",},
     ),html.Div(id='tabs-content-classes')
   ],style={"background": "#111111", 'marginLeft':5,'marginRight':20}
  ) 

@app.callback(Output('tabs-content-classes', 'children'),
              Input('tabs-with-classes', 'value'))

def render_content(value): 
    tab=value
    print(tab)
    if tab == 'Table1':
        print("Table1 OK!")
        return [html.Div([
            html.H1('Демография', style={'textAlign': 'center', 'font-size': '42px', 'color': '#00BFFF'}),                
             html.Div(className="six columns",
                children=[
                   html.Div(
                    children=dcc.Graph(id="pie_graph") 
                )
            ],style={'width':'40.0%','display':'inline-block','vertical-align':'middle',
                     'border':'3px solid','marginLeft':16,'marginRight':0,'marginTop':0,
                     'marginBottom':0, 'padding': '5px 0px 0px 0px'},
          ),
          html.Div(
                    className="twelve columns",
                    children=[
                        html.Div(
                            children=dcc.Graph(id='bottom-bar-graph')
                        )
                    ],style={'width':'58.0%','display':'inline-block','vertical-align':'middle',
                         'border':'3px solid','marginLeft':3,'marginRight':0,'marginTop':0,
                         'marginBottom':0, 'padding': '5px 0px 0px 0px'},
                ),
       ])]
    
    elif tab == 'Table2':
        print("Table2 OK!")
        value='Table2'
        return [html.H1('Популяция',style={'textAlign': "center", 'font-size': '42px', 'color': '#00BFFF'}),
            html.Div(className="six columns",
                children=[html.Div(children=dcc.Graph(id="pop-graph"))
              ],style={'width':'49.5%','display':'inline-block','vertical-align':'middle',
                 'border':'3px solid','marginLeft':16,'marginRight':0,'marginTop':0,
                 'marginBottom':0, 'padding': '5px 0px 0px 0px'},  
            ),                 
            html.Div(className="twelve columns",
                    children=[html.Div(children=dcc.Graph(id='sun_graph'))    
              ],style={'width':'49.5%', 'display':'inline-block',
                             'vertical-align':'middle',
                             'horizontal-align':'middle',                                  
                             'border':'3px solid','marginLeft':3,'marginRight':0,'marginTop':0,
                             'marginBottom':0, 'padding': '5px 0px 0px 0px'},                     
             ),          
        ]
    
    elif tab == 'Table3':
        print("Table3 OK!")
        value='Table3'
        return [html.H1('Возрастное распределение по странам',
                         style={'textAlign': "center", 'font-size': '42px', 'color': '#00BFFF'}),
           html.Div(
            className="six columns",
                children=[
                   html.Div(    
                    children=dcc.Graph(id='median_age_graph')
                   )], style={'width':'49.5%',"height": "610px", 'display':'inline-block','vertical-align':'middle',
                              'border':'3px solid','marginLeft':16,'marginRight':0,'marginTop':0,
                              'marginBottom':0, 'padding': '5px 1px 1px 1px'},                                           
                
          ),
                
            html.Div([        
             dcc.Graph(id='age_graph')
             ],style={'width':'49.5%', "height": "610px", 'display':'inline-block','vertical-align':'middle',
                      'border':'3px solid','marginLeft':3,'marginRight':0,'marginTop':0,
                      'marginBottom':0, 'padding': '5px 1px 1px 1px'},                  
          )                
        ]
     
    elif tab == 'Table4':
        print("Table4 OK!")
        value='Table4'
        return [html.H1("Анализ временных рядов", 
                 style={'textAlign': 'center', 'font-size': '42px', 'color': '#00BFFF'}),
            html.Div(
                className="row",
                children=[
                html.Div(
                  className="twelve columns",
                  children=[
                    html.Div(
                       children=dcc.Graph(id="graph_close"),
                      )
                   ],style={'width':'98.5%', "height": '508px', 
                            'border':'3px solid','marginLeft':16,'marginRight':0,'marginTop':5,
                            'marginBottom':0, 'padding': '1px 1px 1px 2px'},
                    ),
                   ] 
                  ),
        
                html.Div(dash_table.DataTable(
                id='table-end',
                data=FP.to_dict('records'),    
                columns=[
                    {"name": i, "id": i, "deletable": True, "selectable": True} for i in FP.columns
                  ],
                fixed_rows={"headers": True, "data": 4},  
                style_cell={"width": "100px",
                              'backgroundColor': '#111111',
                              'color': 'yellow'          
                             },
                
                style_header={"backgroundColor": "#111111",
                              "color": "yellow",
                              "font-size": "12px",
                              "fontWeight": "bold"
                             },               

                style_table={"height": "250px",
                              "font-size": "18px",
                              "margin": 0,
                              "padding": "8px",
                              "backgroundColor": "#111111"
                              }), style={'width':'99.95%', 'height': '257px',  
                                 'border':'3px solid','marginLeft':3,'marginRight':0,'marginTop':10,
                                 'marginBottom':0, 'padding': '1px 1px 1px 2px'}
               ) 
            ]    
    
@app.callback(Output("pie_graph", "figure"), 
              Input("my-input", "value"))

def drawFigurePie(value):
    ID1=value
    ID = pyperclip.paste()
    df2=df[df['ID1']==ID1]
    figure = px.pie(df2,
    values='POP',
    names='IDNAME', 
    template="plotly_dark", 
    hole=.3,                                    
    height=600, 
    color_discrete_sequence = px.colors.sequential.Plasma_r)
    figure.update_layout(autosize=False, font=dict(family="silom",size=14, color="Yellow"))
    figure.update_layout(title={
        'text': "Prozentuale Verteilung der Bevölkerungszahl in der Region nach Territorium",                                    
        'y':0.99,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        font=dict(family="silom",size=18,color="Yellow"))
 
    return figure

@app.callback(Output("bottom-bar-graph", "figure"), 
              Input("my-input", "value"))

def drawFigureBar2(value):
    ID1=value 
    df1=df[df['ID1']==ID1]
        
    figure = px.bar(df1, x="IDNAME", y="POP", 
    title="Absolute Verteilung der Bevölkerungszahl in der Region nach Territorium",                
    orientation= 'v',
    height=600,                     
    color = "POP", hover_data={'POP': False},
    labels=dict(POP="Anzahl", IDNAME="Territorium"),                
    color_discrete_sequence = px.colors.sequential.Plasma_r).update_layout(
    font=dict(family="silom",
               size=14, 
               color="Yellow"),
    plot_bgcolor = "#111111",
    paper_bgcolor= "#111111")
    figure.update_xaxes(tickangle=45, title_text="Territorium",title_font={"size": 14},
    title_standoff=5, tickfont=dict(family='silom', color='Yellow', size=12))
    figure.update_yaxes(title_text="Anzahl",title_font={"size": 12},title_standoff=5)
    figure.update_layout(title={
        'text': "Absolute Verteilung der Bevölkerungszahl in der Region nach Territorium",                
        'y':0.99,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        font=dict(family="silom",size=18,color="Yellow"))
        
    return figure

@app.callback(Output("pop-graph", "figure"), 
              Input("my-input", "value"))    
def pyramide(value):  
    ID1=value 
    df1=df[df['ID1']==ID1]
    df1=pop[pop['ID']==ID1] 
    men_bins = df1['MKN'].to_numpy()
    women_bins = df1['FKN'].to_numpy()
    
    layout = go.Layout(yaxis=go.layout.YAxis(title='Alter'),
                       xaxis=go.layout.XAxis(
                           range=[-5, 5],
                           tickvals=[-6, 6],
                           title=''
                           ),
                           title="Популяционная пирамида (%)",                
                           plot_bgcolor = "#111111",
                           paper_bgcolor= "#111111",
                           barmode='overlay',
                           bargap=0.1)
    
    
    data = [go.Bar(y=y,
                   x=men_bins,
                   orientation='h',
                   name='Männer',
                   hoverinfo='x',
                   marker=dict(color='powderblue')
                   ),
            go.Bar(y=y,
                   x=women_bins,
                   orientation='h',
                   name='Frauen',
                   text=-1 * women_bins.astype('int'),
                   hoverinfo='text',
                   marker=dict(color='seagreen')
                   )]
    figure = go.Figure(data=data, layout=layout)
    figure.update_layout(autosize=False, 
                         height=600, 
               font=dict(family="silom",size=14, color="Yellow"))
    figure.update_yaxes(title_text="Alter",title_font={"size": 14},
    title_standoff=5, tickfont=dict(family='silom', color='Yellow', size=12))
    figure.update_layout(title={
        'text': "Популяционная пирамида (%)",
        'y':0.99,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        font=dict(family="silom",size=18,color="Yellow"))

    return figure   

@app.callback(Output("sun_graph", "figure"), 
              Input("my-input", "value"))  

def update_sungraph(value):
    ID1=value 
    df1=df[df['ID1']==ID1]
    color_discrete_sequence=['', '#FFAA00', '#00BFFF', '#2D5F91','#819FBD','#819FBD','#91D4D2', '#96BEE6', '#C0D8F0','#E8655F','#F1A39F','#48B7B4']

    fig = px.sunburst(df1, path=["IDNAME", "POP"])
 
    figure =go.Figure(go.Sunburst(
                    labels=fig['data'][0]['labels'].tolist(),
                    parents=fig['data'][0]['parents'].tolist(),
                    marker=dict(colors=color_discrete_sequence)  #funktioniert nicht!   
                  )
                )

    figure.update_layout(title={
                'text': "Популяция по территориям",
                'y':0.99,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                font=dict(family="silom",size=18,color="Yellow"),         
                height=600,
                margin = dict(t=0, l=50, r=0, b=0), 
                plot_bgcolor = "#111111",
                paper_bgcolor= "#111111")
      
    return figure

#Median    
@app.callback(Output('median_age_graph', 'figure'),
              Input("my-input", "value"))  

def plot_median_age(value):
   #https://stackoverflow.com/questions/7844118/how-to-convert-comma-delimited-string-to-list-in-python
    value='Afghanistan,Afghanistan,Kazakhstan,Germany,Japan,United States,Uzbekistan'
    countries = value.split(",") 
    print(countries)
    dfM = age_df[age_df['country'].isin(countries)]
    return {
        'data': [go.Scatter(x=age_df['country'],
                            y=age_df['median_age_total'],
                            mode='markers',
                            showlegend=False,
                            legendgroup='one',
                            name='',
                            hoverlabel={'font': {'size': 20}},
                            marker={'color': '#bbbbbb'})] +
                [go.Scatter(x=dfM[dfM['country']==c]['country'],
                            y=dfM[dfM['country']==c]['median_age_total'],
                            mode='markers',
                            marker={'size': 15},
                            hoverlabel={'font': {'size': 20}},
                            name=c)
                 for c in sorted(countries)],
                        'layout': go.Layout(title={
                            'text': 'Возрастная медиана по странам: ' + ', '.join(countries),                
                            'y':0.99,
                            'x':0.5,
                            'xanchor': 'center',
                            'yanchor': 'top'},
                                         xaxis={
                                   #'title': 'Countries', 
                                   'zeroline': False, 
                                   'showticklabels': False},
                            yaxis={
                                  'zeroline': False},
                            paper_bgcolor="#111111",
                            plot_bgcolor="#111111",
                            height=600, 
                            font=dict(family="silom",
                                      size=18, 
                                      color="Yellow")
       ),
    }
    
@app.callback(Output('age_graph', 'figure'),
             Input("my-input", "value"))   
              
def plot_countries(value):
    value='Afghanistan,Afghanistan,Kazakhstan,Germany,Japan,United States,Uzbekistan'
    countries = value.split(",") 
    print(countries)
    dfM = age_df[age_df['country'].isin(countries)]
          
    return {
        'data': [go.Bar(x=age_categories,
                        y=[0 for i in range(len(age_categories))],
                        showlegend=False,
                        width=0.1,
                        hoverinfo='none')] +
                
                [go.Bar(x=age_categories,
                        y=dfM.iloc[x, 2:7],
                        name=dfM.iloc[x, 0],
                        text=dfM.iloc[x, 2:7].astype(str) + '%',
                        hoverinfo='name+y',
                        textposition='inside',
                        textfont={'color': 'Yellow'})
                 for x in range(len(dfM))],
        'layout': go.Layout(title={
                            'text': 'Возрастное (%) распределение по странам: ' + ', '.join(countries),                
                            'y':0.99,
                            'x':0.5,
                            'xanchor': 'center',
                            'yanchor': 'top'},
                            xaxis={}, 
                            yaxis={}, 
                            barmode='group',
                            plot_bgcolor="#111111",
                            paper_bgcolor="#111111",
                            height=600, 
                            font=dict(family="silom",
                                      size=18, color="Yellow")
                           )
        
    }    

@app.callback(Output('graph_close', 'figure'), 
             Input("my-input", "value"))  
def drawPropeth(value):
    from pandas_datareader import data, wb
    trace_line = go.Scatter(x=list(df_stock.ds),
                                y=list(df_stock.y),
                                name="Aktual",
                                showlegend=True)
   
    pred_line = go.Scatter(x=list(df_pred.ds),
                                y=list(df_pred.yhat_upper),
                                name="Prediction",
                                showlegend=True)
    data = [trace_line, pred_line] 
     
    layout = dict(
        height=500,
        autosize=False,
        plot_bgcolor="#111111",
        paper_bgcolor="#111111",
        legend=dict(x=0.95, y=0.99,
            traceorder="normal",
            title='Aktual & Prognose',
            font=dict(family="silom", size=14, color="yellow"),
            bgcolor='#111111',
            bordercolor='#FFFFFF',
            borderwidth=2
        ),   
        
        yaxis=dict(
        title='Общая заболеваемость',
        titlefont=dict(
            family='silom"',
            size=12,
            color='yellow'
        ),
        showticklabels=True,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=12,
            color='yellow'
        ),
      ),
  
        
        xaxis=dict(
            showticklabels=True,
            tickfont=dict(
                family='Old Standard TT, serif',
                size=12,
                color='yellow'
            ),
            
            showgrid=True, 
            gridcolor='rgb(255, 255, 255)',
            gridwidth=1,       
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                         label='YTD',
                         step='year',
                         stepmode='todate'),
                    dict(count=1,
                         label='1y',
                         step='year',
                         stepmode='backward'),
                    dict(step='all')
                ])
            ),
            rangeslider=dict(visible=True),
            type='date'
        ) 
    )
    return {"data": data, "layout": layout}
                       
if __name__ == '__main__':
    app.run_server(debug=False, port=8085)


Dash is running on http://127.0.0.1:8085/



 * Running on http://127.0.0.1:8085
Press CTRL+C to quit


OSError: Address 'http://127.0.0.1:8085' already in use.
    Try passing a different port to run_server.